# Optimisation of commerical building energy system

Model of a large commercial building (100kW mean load is pretty hefty)

Setup stuff

In [1]:
from tqdm.auto import tqdm
import numpy as np
from numpy import random
from scipy import stats
from models import run_model
from functools import partial
from utils import get_Gurobi_WLS_env, fmt_design_results

In [2]:
env = get_Gurobi_WLS_env(silence=True)
run_model = partial(run_model, env=env)

In [3]:
solar_years = range(2012,2018)
load_years = range(2012,2018)

In [4]:
design_options = [
    [500,200],
    [500,400],
    [600,500],
    [750,600],
    [750,800]
]

In [5]:
costs = []
for design in tqdm(design_options):
    costs.append(run_model(*design)['total'])

  0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
print(costs)

[170403.00530292297, 172220.64528958552, 170352.38107085635, 168396.4078633337, 175434.6180990048]


In [7]:
print(min(costs))
print(design_options[np.argmin(costs)])

168396.4078633337
[750, 600]


In [8]:
cfact_cost = run_model(0, 0)['total']
print(cfact_cost)

214167.54841474074


Stochastic optimization ...

In [9]:
avg_costs = []
for design in tqdm(design_options):
    avg_cost = np.mean([run_model(*design, solar_year=[year])['total'] for year in tqdm(solar_years,leave=False)])
    avg_costs.append(avg_cost)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
print(min(avg_costs))
print(design_options[np.argmin(avg_costs)])

166619.95467501873
[750, 600]


In [11]:
print(min(avg_costs) - min(costs))
print((min(avg_costs) - min(costs))/min(avg_costs) * 100)

-1776.4531883149757
-1.0661707307386026


LP ...

In [12]:
LP_results = run_model(None,None)
print(fmt_design_results(LP_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       834.6
Battery capacity  kWh       343.3
Total cost        £k/yr     161.7
CAPEX             £k/yr      90.8
OPEX              £k/yr      70.9


Improvement from LP vs static options

In [13]:
print(LP_results['total'] - min(costs))
print((LP_results['total'] - min(costs))/min(costs)*100)

-6713.300381640845
-3.9866054548439007


SP ...

In [14]:
SP_results = run_model(None,None,solar_year=list(range(2012,2018)))
print(fmt_design_results(SP_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       794.8
Battery capacity  kWh       340.7
Total cost        £k/yr     160.5
CAPEX             £k/yr      87.4
OPEX              £k/yr      73.1


In [15]:
print((SP_results['total'] - min(avg_costs))/min(avg_costs)*100)

-3.673709122439898


Stochastic optimisation with lots of uncertainties ...

In [23]:
nsamples = 10
scenarios = {
    'solar_year': random.choice(solar_years, nsamples),
    'load_year': random.choice(load_years, nsamples),
    'mean_load': stats.truncnorm.rvs(-2, 2, loc=100, scale=10, size=nsamples),
    'battery_efficiency': stats.truncnorm.rvs(-2, 2, loc=0.95, scale=0.05, size=nsamples),
    'battery_cost': stats.truncnorm.rvs(-2, 2, loc=70, scale=5, size=nsamples),
}

In [24]:
avg_costs = []
for design in tqdm(design_options):
    avg_cost = np.mean([run_model(
        *design,
        solar_year=[scenarios['solar_year'][i]],
        load_year=[scenarios['load_year'][i]],
        mean_load=[scenarios['mean_load'][i]],
        battery_efficiency=[scenarios['battery_efficiency'][i]],
        battery_cost=[scenarios['battery_cost'][i]]
        )['total']
            for i in tqdm(range(nsamples),leave=False)])
    avg_costs.append(avg_cost)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [25]:
print(min(avg_costs))
print(design_options[np.argmin(avg_costs)])

162491.5696527055
[750, 600]


In [26]:
SP_results = run_model(
    solar_capacity=None,
    battery_capacity=None,
    solar_year=scenarios['solar_year'],
    load_year=scenarios['load_year'],
    mean_load=scenarios['mean_load'],
    battery_efficiency=scenarios['battery_efficiency'],
    battery_cost=scenarios['battery_cost']
)

In [27]:
print(fmt_design_results(SP_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       765.2
Battery capacity  kWh       327
Total cost        £k/yr     157.2
CAPEX             £k/yr      84.3
OPEX              £k/yr      72.8


Could also then look at benefit of continuous solution (i.e. SP vs discrete stochastic), and Value of Stochastic Solution